In [17]:
import pandas as pd
import numpy as np

dft = pd.read_csv("complete_times.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [18]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [19]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [20]:
import datetime
def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [42]:
gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")

martedi: primo : Cremona, Felice Cavallotti, 2 : 56.6
martedi: primo : Casalpusterlengo : 37.5
martedi: primo : Cremona, Ugolani Dati, 4 : 17.950000000000003
martedi: primo : Crema, Civerchi : -7.533333333333331
martedi: failed : 45.38333333333333 : 5.0 : -2.5333333333333314
martedi: failed : 113.08333333333333 : 10.0 : 7.466666666666669
martedi: failed : 114.38333333333334 : 10.0 : 17.46666666666667
martedi: failed : 315.3 : 20.0 : 37.46666666666667
martedi: failed : 275.6666666666667 : 10.0 : 47.46666666666667
martedi: primo : Offanengo : 7.616666666666667
martedi: failed : 65.25 : 5.0 : 12.616666666666667
martedi: primo : Gussola : 14.433333333333334
martedi: primo : Solarolo : 170.35
martedi: primo : Ostiano : 146.71666666666667
martedi: primo : Scandolara Ravara, Italia : 148.4
martedi: primo : Persico Dosimo : 119.36666666666667
martedi: primo : Casalmaggiore : 89.25
martedi: primo : Gadesco : 72.8
martedi: primo : Viadana : 23.96666666666666
martedi: primo : Pescarolo : -6.19999

In [43]:
impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")

martedi: impossible : Bonemerse : 5.0  27.616666666666667
martedi: impossible : Sospiro : 10.0  112.08333333333333
martedi: impossible : Vailate : 10.0  41.583333333333336
martedi: impossible : Pianengo : 20.0  279.01666666666665
martedi: secondo : Offanengo : 9.083333333333334


In [21]:
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='martedi'
#Liste dei due furgoni
f1 = []
f2 = []
#indici
i1 = -1
i2 = -1
#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start
row2 = start
#Lista dei paesi non raggiungibili
impossibile = []
for i in range(len(time_table[day].dropna())):
    paese=table[day][i]
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]
    value2=row2.loc[:,get_column(dft, paese).values]
    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1 and i2 != -1):
        diff1=difference(day, i1, i)
        diff2=difference(day, i2, i)
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        if(i2 == -1):
            diff2 = value2.values[0][0]
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5%) si sceglie il migliore
    if(float(value1.values[0][0])<= float(diff1) + (float(diff1)/20) or float(value2.values[0][0])<= float(diff2) + float(diff2)/20):
        if(value1.values[0][0] <= value2.values[0][0]):
            f1.append(i)
            i1 = i
            row1 = dft[get_row(dft, table[day][i]).values]
        else:
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
        impossibile.append(i)
print(f1)
print(f2)
print(impossibile)

ValueError: could not convert string to float: '3.396.666.666.666.660'

In [16]:
dft[get_row(dft, "bonemerse").values].loc[:, get_column(dft, "sospiro").values]

,"Sospiro, Europa, 2"
"Bonemerse, Roma, 25",113.08333333333333
